# **AMAZON SALE** **DATA**

In [ ]:
!pip install pyspark

In [ ]:
import pyspark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import*
from pyspark.sql.types import*

In [ ]:
spark=SparkSession.builder.appName("Dataframe").getOrCreate()

In [ ]:
spark

# Read .CSV file format



In [ ]:
df=spark.read.csv("/content/AmazonSalesData.csv", header= True, inferSchema=True)

In [ ]:
df.printSchema()

root
 |-- Region: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Item Type: string (nullable = true)
 |-- Sales Channel: string (nullable = true)
 |-- Order Priority: string (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Order ID: integer (nullable = true)
 |-- Ship Date: string (nullable = true)
 |-- Units Sold: integer (nullable = true)
 |-- Unit Price: double (nullable = true)
 |-- Unit Cost: double (nullable = true)
 |-- Total Revenue: double (nullable = true)
 |-- Total Cost: double (nullable = true)
 |-- Total Profit: double (nullable = true)



In [ ]:
df.show()

+--------------------+--------------------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
|              Region|             Country|      Item Type|Sales Channel|Order Priority|Order Date| Order ID| Ship Date|Units Sold|Unit Price|Unit Cost|Total Revenue|Total Cost|Total Profit|
+--------------------+--------------------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
|Australia and Oce...|              Tuvalu|      Baby Food|      Offline|             H| 5/28/2010|669165933| 6/27/2010|      9925|    255.28|   159.42|    2533654.0| 1582243.5|    951410.5|
|Central America a...|             Grenada|         Cereal|       Online|             C| 8/22/2012|963881480| 9/15/2012|      2804|     205.7|   117.11|     576782.8| 328376.44|   248406.36|
|              Europe|              Russia|Of

# Select Col


In [ ]:
df.select(col("Region"), col('country')).show()

+--------------------+--------------------+
|              Region|             country|
+--------------------+--------------------+
|Australia and Oce...|              Tuvalu|
|Central America a...|             Grenada|
|              Europe|              Russia|
|  Sub-Saharan Africa|Sao Tome and Prin...|
|  Sub-Saharan Africa|              Rwanda|
|Australia and Oce...|     Solomon Islands|
|  Sub-Saharan Africa|              Angola|
|  Sub-Saharan Africa|        Burkina Faso|
|  Sub-Saharan Africa|Republic of the C...|
|  Sub-Saharan Africa|             Senegal|
|                Asia|          Kyrgyzstan|
|  Sub-Saharan Africa|          Cape Verde|
|                Asia|          Bangladesh|
|Central America a...|            Honduras|
|                Asia|            Mongolia|
|              Europe|            Bulgaria|
|                Asia|           Sri Lanka|
|  Sub-Saharan Africa|            Cameroon|
|                Asia|        Turkmenistan|
|Australia and Oce...|          

## Region and country with profit


In [ ]:
region_and_country_with_profit = df.groupBy(col('Region')).sum('Total Profit')
region_and_country_with_profit.show()


+--------------------+--------------------+
|              Region|   sum(Total Profit)|
+--------------------+--------------------+
|Middle East and N...|   5761191.859999999|
|Australia and Oce...|          4722160.03|
|              Europe|1.1082938629999999E7|
|  Sub-Saharan Africa|1.2183211400000004E7|
|Central America a...|          2846907.85|
|       North America|          1457942.76|
|                Asia|          6113845.87|
+--------------------+--------------------+



## Region with highest profit


In [ ]:
region_with_highest_profit = region_and_country_with_profit.orderBy("sum(Total Profit)", ascending=False).first()


# Region has the highest profit with total profit


In [ ]:
print(f"{region_with_highest_profit['Region']} Region has the highest profit with a total profit of {region_with_highest_profit['sum(Total Profit)']}")

Sub-Saharan Africa Region has the highest profit with a total profit of 12183211.400000004


## Country with highest profit


In [ ]:
country_with_highest_profit = df.filter(col('Region') == region_with_highest_profit['Region']) \
                                .groupBy(col('Country')).sum('Total Profit') \
                                .orderBy("sum(Total Profit)", ascending=False)
country_with_highest_profit.show()

+------------+-----------------+
|     Country|sum(Total Profit)|
+------------+-----------------+
|    Djibouti|       2425317.87|
|     Myanmar|        1802771.7|
|    Pakistan|       1719922.04|
|      Samoa |       1678540.98|
|    Honduras|       1609947.52|
|     Iceland|       1541705.29|
|  Azerbaijan|       1512926.83|
| Switzerland|       1512729.45|
|      Mexico|       1457942.76|
|      Rwanda|       1417493.49|
|  The Gambia|       1385883.27|
|     Romania|        1375311.7|
|Turkmenistan|        1267258.4|
|   Sri Lanka|       1208744.24|
|        Iran|       1128242.43|
|   Lithuania|       1046233.75|
|      Tuvalu|         951410.5|
|  Mozambique|        889472.91|
|      Brunei|         846885.0|
|      Monaco|        825738.04|
+------------+-----------------+
only showing top 20 rows



## Country under the region has the highest profit

In [ ]:
print(f"{country_with_highest_profit['Country']} country, under region {region_with_highest_profit['Region']} has the highest profit with a total profit of {country_with_highest_profit['sum(Total Profit)']}")


Djibouti country, under region Sub-Saharan Africa has the highest profit with a total profit of 2425317.87


# Window functions

In [ ]:
from pyspark.sql.window import Window

In [ ]:
df.withColumn('sum of total profit', sum('Total profit').over(Window.orderBy('Region'))).show()

+--------------------+--------------------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+-------------------+
|              Region|             Country|      Item Type|Sales Channel|Order Priority|Order Date| Order ID| Ship Date|Units Sold|Unit Price|Unit Cost|Total Revenue|Total Cost|Total Profit|sum of total profit|
+--------------------+--------------------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+-------------------+
|                Asia|          Kyrgyzstan|     Vegetables|       Online|             H| 6/24/2011|814711606| 7/12/2011|       124|    154.06|    90.93|     19103.44|  11275.32|     7828.12|         6113845.87|
|                Asia|          Bangladesh|        Clothes|       Online|             L| 1/13/2017|187310731|  3/1/2017|      8263|    109.28|    35.84|    

In [ ]:
total = df.withColumn('sum of total profit', sum('Total profit').over(Window.partitionBy('Region')))
total.show()

+--------------------+--------------------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+-------------------+
|              Region|             Country|      Item Type|Sales Channel|Order Priority|Order Date| Order ID| Ship Date|Units Sold|Unit Price|Unit Cost|Total Revenue|Total Cost|Total Profit|sum of total profit|
+--------------------+--------------------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+-------------------+
|                Asia|          Kyrgyzstan|     Vegetables|       Online|             H| 6/24/2011|814711606| 7/12/2011|       124|    154.06|    90.93|     19103.44|  11275.32|     7828.12|         6113845.87|
|                Asia|          Bangladesh|        Clothes|       Online|             L| 1/13/2017|187310731|  3/1/2017|      8263|    109.28|    35.84|    

In [ ]:
total.show()

+--------------------+--------------------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+-------------------+
|              Region|             Country|      Item Type|Sales Channel|Order Priority|Order Date| Order ID| Ship Date|Units Sold|Unit Price|Unit Cost|Total Revenue|Total Cost|Total Profit|sum of total profit|
+--------------------+--------------------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+-------------------+
|                Asia|          Kyrgyzstan|     Vegetables|       Online|             H| 6/24/2011|814711606| 7/12/2011|       124|    154.06|    90.93|     19103.44|  11275.32|     7828.12|         6113845.87|
|                Asia|          Bangladesh|        Clothes|       Online|             L| 1/13/2017|187310731|  3/1/2017|      8263|    109.28|    35.84|    

In [ ]:
df.show()

+--------------------+--------------------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
|              Region|             Country|      Item Type|Sales Channel|Order Priority|Order Date| Order ID| Ship Date|Units Sold|Unit Price|Unit Cost|Total Revenue|Total Cost|Total Profit|
+--------------------+--------------------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
|Australia and Oce...|              Tuvalu|      Baby Food|      Offline|             H| 5/28/2010|669165933| 6/27/2010|      9925|    255.28|   159.42|    2533654.0| 1582243.5|    951410.5|
|Central America a...|             Grenada|         Cereal|       Online|             C| 8/22/2012|963881480| 9/15/2012|      2804|     205.7|   117.11|     576782.8| 328376.44|   248406.36|
|              Europe|              Russia|Of

## The highest number of orders placed within a 72-hour period

In [ ]:
orders = df.groupBy(window(col('Order Date'), '72 hours'))\
         .agg(count('Order ID').alias('count_order_id'))\
         .orderBy('count_order_id', ascending=False)


print(f"Highest number of orders ordered under a 72 hr period is {orders.first()}")


Highest number of orders ordered under a 72 hr period is None


## Average number of units sold in a month

In [ ]:
df.groupBy(month('Order Date').alias('month')) \
                   .agg(avg(col('Units Sold')).alias('avg_units_sold')) \
                   .orderBy('avg_units_sold', ascending=False).first()


Row(month=None, avg_units_sold=5128.71)

## Margin

In [ ]:
margin = df.withColumnsRenamed({'Unit Price':'SP','Unit Cost':'CP'})
margin.show()



+--------------------+--------------------+---------------+-------------+--------------+----------+---------+----------+----------+------+------+-------------+----------+------------+
|              Region|             Country|      Item Type|Sales Channel|Order Priority|Order Date| Order ID| Ship Date|Units Sold|    SP|    CP|Total Revenue|Total Cost|Total Profit|
+--------------------+--------------------+---------------+-------------+--------------+----------+---------+----------+----------+------+------+-------------+----------+------------+
|Australia and Oce...|              Tuvalu|      Baby Food|      Offline|             H| 5/28/2010|669165933| 6/27/2010|      9925|255.28|159.42|    2533654.0| 1582243.5|    951410.5|
|Central America a...|             Grenada|         Cereal|       Online|             C| 8/22/2012|963881480| 9/15/2012|      2804| 205.7|117.11|     576782.8| 328376.44|   248406.36|
|              Europe|              Russia|Office Supplies|      Offline|       

## Selection of col with margin Df

In [ ]:
margin.select('order id','SP','CP').show()

+---------+------+------+
| order id|    SP|    CP|
+---------+------+------+
|669165933|255.28|159.42|
|963881480| 205.7|117.11|
|341417157|651.21|524.96|
|514321792|  9.33|  6.92|
|115456712|651.21|524.96|
|547995746|255.28|159.42|
|135425221|668.27|502.54|
|871543967|154.06| 90.93|
|770463311| 81.73| 56.67|
|616607081| 205.7|117.11|
|814711606|154.06| 90.93|
|939825713|109.28| 35.84|
|187310731|109.28| 35.84|
|522840487|668.27|502.54|
|832401311| 81.73| 56.67|
|972292029|109.28| 35.84|
|419123971| 437.2|263.33|
|519820964| 47.45| 31.79|
|441619336|668.27|502.54|
|322067916|421.89|364.69|
+---------+------+------+
only showing top 20 rows



## Margin Percentage

In [ ]:
margin_percentage = margin.withColumn("Percentage", round(((col('SP') - col('CP')) / col('SP')) * 100))
margin_percentage.show()

+--------------------+--------------------+---------------+-------------+--------------+----------+---------+----------+----------+------+------+-------------+----------+------------+----------+
|              Region|             Country|      Item Type|Sales Channel|Order Priority|Order Date| Order ID| Ship Date|Units Sold|    SP|    CP|Total Revenue|Total Cost|Total Profit|Percentage|
+--------------------+--------------------+---------------+-------------+--------------+----------+---------+----------+----------+------+------+-------------+----------+------------+----------+
|Australia and Oce...|              Tuvalu|      Baby Food|      Offline|             H| 5/28/2010|669165933| 6/27/2010|      9925|255.28|159.42|    2533654.0| 1582243.5|    951410.5|      38.0|
|Central America a...|             Grenada|         Cereal|       Online|             C| 8/22/2012|963881480| 9/15/2012|      2804| 205.7|117.11|     576782.8| 328376.44|   248406.36|      43.0|
|              Europe|   

## Lowest and Highest of margin percentage

In [ ]:
lowest_margin_percentage = margin_percentage.orderBy('Percentage', ascending=True)
highest_margin_percentage = margin_percentage.orderBy('Percentage', ascending=False)
lowest_margin_percentage.show(10)
highest_margin_percentage.show(10)

+--------------------+------------+---------------+-------------+--------------+----------+---------+----------+----------+------+------+-------------+----------+------------+----------+
|              Region|     Country|      Item Type|Sales Channel|Order Priority|Order Date| Order ID| Ship Date|Units Sold|    SP|    CP|Total Revenue|Total Cost|Total Profit|Percentage|
+--------------------+------------+---------------+-------------+--------------+----------+---------+----------+----------+------+------+-------------+----------+------------+----------+
|Australia and Oce...|  East Timor|           Meat|       Online|             L| 7/31/2012|322067916| 9/11/2012|      5908|421.89|364.69|   2492526.12|2154588.52|    337937.6|      14.0|
|  Sub-Saharan Africa|  The Gambia|           Meat|       Online|             M| 1/14/2017|825304400| 1/23/2017|      4767|421.89|364.69|   2011149.63|1738477.23|    272672.4|      14.0|
|              Europe|      Russia|Office Supplies|      Offline|

In [ ]:
df.show()

+--------------------+--------------------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
|              Region|             Country|      Item Type|Sales Channel|Order Priority|Order Date| Order ID| Ship Date|Units Sold|Unit Price|Unit Cost|Total Revenue|Total Cost|Total Profit|
+--------------------+--------------------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
|Australia and Oce...|              Tuvalu|      Baby Food|      Offline|             H| 5/28/2010|669165933| 6/27/2010|      9925|    255.28|   159.42|    2533654.0| 1582243.5|    951410.5|
|Central America a...|             Grenada|         Cereal|       Online|             C| 8/22/2012|963881480| 9/15/2012|      2804|     205.7|   117.11|     576782.8| 328376.44|   248406.36|
|              Europe|              Russia|Of

## Top 3 highest-value order for a specific Item type

In [ ]:
window_spec = Window.partitionBy(col('Item Type')).orderBy(col('Total Profit').desc())
highest_valuable_orders = df.withColumn("row_id", row_number().over(window_spec)) \
                            .filter(col('row_id').isin(1, 2, 3))

highest_valuable_orders.show(15)

+--------------------+--------------------+---------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+------+
|              Region|             Country|Item Type|Sales Channel|Order Priority|Order Date| Order ID| Ship Date|Units Sold|Unit Price|Unit Cost|Total Revenue|Total Cost|Total Profit|row_id|
+--------------------+--------------------+---------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+------+
|Australia and Oce...|              Tuvalu|Baby Food|      Offline|             H| 5/28/2010|669165933| 6/27/2010|      9925|    255.28|   159.42|    2533654.0| 1582243.5|    951410.5|     1|
|              Europe|              Monaco|Baby Food|      Offline|             H| 5/29/2012|688288152|  6/2/2012|      8614|    255.28|   159.42|   2198981.92|1373243.88|   825738.04|     2|
|              Europe|              Norw

## Item mostly frequently sold on weekends

In [ ]:
Item_most_frequently_sold = df.filter((col("Region")=="Sub-Saharan Africa") & (weekday(col("Ship Date"))).isin([5,6]))\
                            .groupBy(col("Item Type")).agg(sum('units sold').alias("sum_units_sold"))\
                            .orderBy("sum_units_sold",ascending=False).first()


## Revenue by year

In [ ]:
revenue_by_year = df.groupBy(year(col("Order Date")).alias("Year")) \
                     .agg(sum("Total Revenue").alias("Yearly Revenue"))

revenue_by_year.show()


+----+--------------------+
|Year|      Yearly Revenue|
+----+--------------------+
|NULL|1.3734876830999997E8|
+----+--------------------+



In [ ]:
windowSpec = Window.orderBy("Year")
revenue_with_growth = revenue_by_year.withColumn("YoY Growth %",concat(round(((col("Yearly Revenue") - lag("Yearly Revenue", 1).over(windowSpec)) /
                                                                              lag("Yearly Revenue", 1).over(windowSpec))*100).cast("int").cast("string"),lit(" %")))
revenue_with_growth.show()

+----+--------------------+------------+
|Year|      Yearly Revenue|YoY Growth %|
+----+--------------------+------------+
|NULL|1.3734876830999997E8|        NULL|
+----+--------------------+------------+



In [ ]:
df.write.format('parquet').mode('append').save("/content/AmazonSalesDataNotebook.csv")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')